In [3]:
"""
Utility Functions and Constants
================================
Shared helpers for the NTH-ED Triage Lead Tool.

Contains:
- Color schemes (consistent across all visualizations)
- Formatting functions
- Common constants
"""

# =============================================================================
# COLOR SCHEMES
# =============================================================================

# Triage colors (clinical standard: red=critical, green=minor)
TRIAGE_COLORS = {
    '1-RESUSCITATION': '#DC2626',   # Red - most critical
    '2-EMERGENCY': '#F97316',        # Orange
    '3-URGENT': '#EAB308',           # Yellow
    '4-LESS URGENT': '#22C55E',      # Green
    '5-NON-URGENT': '#3B82F6',       # Blue - least urgent
}

# Zone colors
ZONE_COLORS = {
    'Resus': '#DC2626',      # Red - resuscitation
    'Red': '#DC2626',        # Red zone
    'YZ': '#EAB308',         # Yellow zone
    'GZ': '#22C55E',         # Green zone  
    'EPZ': '#8B5CF6',        # Purple - psychiatric
    'SA': '#F97316',         # Orange - sub-acute
    'A': '#3B82F6',          # Blue - area A
    'HH': '#EC4899',         # Pink
    'Unknown': '#6B7280',    # Gray
    'Checkout': '#6B7280',   # Gray
}

# Risk level colors (for ML predictions)
RISK_COLORS = {
    'high': '#DC2626',       # Red
    'medium': '#F97316',     # Orange
    'low': '#22C55E',        # Green
}

# Dashboard theme colors
THEME = {
    'primary': '#1E40AF',    # Dark blue
    'secondary': '#6B7280',  # Gray
    'success': '#22C55E',    # Green
    'warning': '#F97316',    # Orange
    'danger': '#DC2626',     # Red
    'info': '#3B82F6',       # Blue
    'background': '#F9FAFB', # Light gray
    'card_bg': '#FFFFFF',    # White
}


# =============================================================================
# CLINICAL CONSTANTS
# =============================================================================

# Standard protocol for conformance checking
STANDARD_PROTOCOL = ['Triage', 'Registration', 'Assessment', 'Discharge']

# Expected events
ALL_EVENTS = [
    'Ambulance Arrival',
    'Triage',
    'Registration',
    'Ambulance Transfer',
    'Assessment',
    'Consult Request',
    'Consult Arrival',
    'Discharge',
    'Left ED'
]

# PIA targets by CTAS level (in minutes) - based on Canadian guidelines
PIA_TARGETS = {
    '1-RESUSCITATION': 0,     # Immediate
    '2-EMERGENCY': 15,        # 15 minutes
    '3-URGENT': 30,           # 30 minutes  
    '4-LESS URGENT': 60,      # 60 minutes
    '5-NON-URGENT': 120,      # 120 minutes
}


# =============================================================================
# FORMATTING FUNCTIONS
# =============================================================================

def format_minutes(minutes):
    """Convert minutes to human-readable format (e.g., '2h 30m')"""
    if minutes is None or pd.isna(minutes):
        return "N/A"
    
    hours = int(minutes // 60)
    mins = int(minutes % 60)
    
    if hours > 0:
        return f"{hours}h {mins}m"
    else:
        return f"{mins}m"


def format_percentage(value, decimals=1):
    """Format decimal as percentage string"""
    if value is None or pd.isna(value):
        return "N/A"
    return f"{value * 100:.{decimals}f}%"


def format_number(value):
    """Format number with thousands separator"""
    if value is None or pd.isna(value):
        return "N/A"
    return f"{value:,.0f}"


def get_risk_level(probability, high_threshold=0.7, medium_threshold=0.4):
    """Convert probability to risk level category"""
    if probability >= high_threshold:
        return 'high'
    elif probability >= medium_threshold:
        return 'medium'
    else:
        return 'low'


def get_risk_color(probability, high_threshold=0.7, medium_threshold=0.4):
    """Get color based on risk probability"""
    level = get_risk_level(probability, high_threshold, medium_threshold)
    return RISK_COLORS[level]


# =============================================================================
# DATA VALIDATION
# =============================================================================

def validate_event_log(df):
    """
    Validate that event log has required columns.
    Returns (is_valid, error_message)
    """
    required_cols = ['case_id', 'activity', 'timestamp']
    
    missing = [col for col in required_cols if col not in df.columns]
    
    if missing:
        return False, f"Missing required columns: {missing}"
    
    return True, "Valid"


def validate_visits(df):
    """
    Validate that visits dataframe has required columns.
    Returns (is_valid, error_message)
    """
    required_cols = ['case_id', 'triage_level', 'initial_zone', 'pia_minutes']
    
    missing = [col for col in required_cols if col not in df.columns]
    
    if missing:
        return False, f"Missing required columns: {missing}"
    
    return True, "Valid"


# Need pandas for some functions
import pandas as pd